# Trabajo Final 

_Fecha: 16 de mayo 2019_

Enviar un enlace a un repositorio de GitHub que contenga los trabajos realizados durante el semestre, la carpeta debe contener las siguientes entregas:

+ MNIST (digitos) con algoritmo deterministico
+ Tic-tac-toe
+ Sistema experto
+ Opcional (Artificial Life)
+ MNIST (digitos) machine learning - Kmeans
+ MNIST (digitos) deep learning, Tensorflow

# MNIST (dígitos) deep learning, Tensorflow

__Integrantes:__

+ Nombre: **Brayan G. Bejarano**
+ Nombre: **Bayron Campaz**


Entrenar una red neuronal con el fin de detectar los dígitos de MNIST y comparar sus resultados contra el algoritmo determinístico y el modelo de machine learning kmeans.

Evaluar los resultados contra dos medidas de evaluación (accuracy y tiempo de entramiento). Se recomienda realizar el hold-out con un 80% para entrenamiento y un conjunto de testeo del 20%.


## Comparación contra el mejor modelo TensorFlow

La siguiente es una implementacion con TensorFlow para la prediccion de numeros como un conjunto de imagenes.

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Read data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [27]:
def layer(input, weigth_shape, bias_shape):
    w_init = tf.random_normal_initializer(stddev= (2.0/weigth_shape[0]) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weigth_shape, initializer = w_init)
    b = tf.get_variable("b", bias_shape, initializer = bias_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

Se generan las capas ocultas y la capa de salida, utilizamos 2 capas ocultas para mayor precision

In [28]:
def inference(x):     
    with tf.variable_scope("hidden_1"):
        hidden_1 = layer(x , [784, 256], [256])
    with tf.variable_scope("hidden_2"):
        hidden_2 = layer(hidden_1, [256, 256], [256])
    with tf.variable_scope("output"):
        output = layer(hidden_2, [256, 10], [10])
    return output

In [29]:
def loss(output, y):
    xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
    loss = tf.reduce_mean(xentropy)
    return loss

Usamos algunos metodos para la aproximacion por regresion logistica

In [30]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("validation", accuracy)
    return accuracy

Entrenamos al algoritmo

In [31]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [32]:
%%time
import tensorflow as tf
# Parameters

learning_rate = 0.01
training_epochs = 300
batch_size = 100
display_step = 5
train_losses, test_losses = [], []


with tf.Graph().as_default():
    # mnist data image of shape 28*28=784
    x = tf.placeholder(tf.float32, [None, 784])    
    # 0-9 digits recognition => 10 classes
    y = tf.placeholder(tf.float32, [None, 10])    
    output = inference(x)    
    cost = loss(output, y) 
    
    global_step = tf.Variable(0, name='global_step', trainable=False)    
    train_op = training(cost, global_step)    
    eval_op = evaluate(output, y)
    
    summary_op = tf.summary.merge_all()    
    saver = tf.train.Saver()
    
    sess = tf.Session()    
    summary_writer = tf.summary.FileWriter("mnist_logs/", graph=sess.graph)
    
    init_op = tf.global_variables_initializer()    
    sess.run(init_op)
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            feed_dict = {x : mbatch_x, y : mbatch_y}
            sess.run(train_op, feed_dict=feed_dict)
            # Compute average loss
            minibatch_cost = sess.run(cost, feed_dict=feed_dict)
            avg_cost += minibatch_cost/total_batch
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            val_feed_dict = {
                x : mnist.validation.images,
                y : mnist.validation.labels
            }
            accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
            print ("Validation Error:", (1 - accuracy))
            train_losses.append(1 - accuracy)
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, sess.run(global_step))
            
            saver.save(sess, "mnist_logs/model-checkpoint", global_step=global_step)
            
    print("Optimization Finished!")
    
    test_feed_dict = {
        x : mnist.test.images,
        y : mnist.test.labels
    }
    accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
    #writer = tf.summary.FileWriter("logistic_logs3/", graph=sess.graph)
    #writer.add_graph(tf.get_default_graph())
    print("Test Accuracy:", accuracy)

AttributeError: module 'tensorflow' has no attribute 'Graph'

Como se observa se tiene una precision de alrededor de 97.8% la cual es bastante alta, eso si luego de que  el modelo estuviese alrededor de 45 minutos de entrenamiento.

## Comparación contra el mejor modelo KMeans

La siguiente es una implementacion que usa el algoritmo KNN desde la libreria sklearn para la prediccion de numeros como un conjunto de imagenes.

In [16]:
from sklearn.datasets import load_digits
import sklearn.metrics as metrics
import pylab as pl
import sklearn
import math

En las siguientes lineas se hace la division del conjunto de datos con un porcentaje del 80% para entrenamiento y un 20% para pruebas.

In [18]:
digits = load_digits()
percentFit = 0.8
numImagenes = len(digits.images)
limitInferior =  math.floor(numImagenes*percentFit)+1
limitSuperior = math.ceil(numImagenes*percentFit)

In [7]:
y = digits.target[0 : limitInferior] # El método nos provee las etiquetas de las imágenes en un arreglo
w = digits.target[limitSuperior :]

Para simplificar la representación de los datos, se aplana las matrices de pixeles para poner cada imagen en un solo array de 64 pixeles:

In [19]:
X = digits.images[0 : limitInferior].reshape((len(y), -1)) # Se selecciona el porcentaje de entrenamiento.
Z = digits.images[limitSuperior :].reshape((len(w),-1))

Se importa las librerias correspondientes a KNN.

In [20]:
#from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
import pylab as plt

Se realiza el entrenamiento.

In [53]:
%%time
knn = KNeighborsClassifier(4)
fit = knn.fit(X, y)

Wall time: 9 ms


Luego procedemos a estimar con el conjuto de datos destinado para tests.

In [54]:
%%time
w_estimado = fit.predict(Z)

Wall time: 166 ms


Finalmente calculamos la eficacia del algoritma lo cual nos da:

In [55]:
metrics.accuracy_score(w, w_estimado)

0.9665738161559888

Lo anterior indica que el algoritmo KNN con los datos de entrenamiento brindados y 4 nodos predice el numero correctamente con un 96% de efectividad, lo que se puede considerar como un valor bastante alto.

# Comparación contra el resultado del algoritmo deterministico

## Código para el 4.

Se obtiene los digitos y se almacenan en un arreglo los indices en el que el número objetivo es igual a 4 y reciprocamente se almacenan en otro arreglo los que son diferentes de 4.

In [4]:
from sklearn.datasets import load_digits
import sklearn.metrics as metrics
import pylab as pl
import sklearn

digits = load_digits()

theFours = []
theNotFours = []
i: int = 0
numFours: int = 0
     
while i < len(digits.target):
    if digits.target[i] == 4:
        theFours.append(i)
        numFours = numFours + 1
    else:
        theNotFours.append(i)
    i = i + 1
print(theFours)

[4, 14, 24, 41, 64, 68, 87, 97, 100, 110, 111, 121, 124, 134, 144, 154, 171, 194, 198, 225, 228, 238, 239, 247, 250, 260, 270, 280, 297, 320, 324, 343, 353, 356, 366, 367, 377, 380, 390, 400, 410, 427, 450, 454, 473, 483, 486, 496, 497, 507, 510, 520, 530, 540, 557, 580, 584, 603, 613, 616, 626, 627, 637, 640, 650, 660, 670, 687, 710, 714, 733, 743, 746, 756, 757, 767, 770, 780, 790, 800, 817, 840, 844, 863, 873, 876, 886, 887, 897, 900, 909, 919, 929, 946, 966, 970, 988, 998, 1001, 1011, 1012, 1022, 1023, 1033, 1043, 1053, 1070, 1091, 1095, 1114, 1124, 1127, 1137, 1138, 1148, 1151, 1161, 1171, 1181, 1198, 1221, 1225, 1244, 1254, 1257, 1267, 1268, 1278, 1281, 1291, 1301, 1311, 1328, 1351, 1355, 1374, 1384, 1387, 1397, 1398, 1408, 1411, 1419, 1429, 1439, 1456, 1479, 1483, 1502, 1512, 1515, 1525, 1526, 1536, 1539, 1549, 1559, 1567, 1584, 1607, 1611, 1628, 1638, 1641, 1651, 1652, 1660, 1661, 1671, 1681, 1691, 1708, 1731, 1735, 1754, 1764, 1767, 1777, 1778, 1788, 1791]


El siguiente metodo contiene las reglas y retricciones que determinan si la matriz de entrada representa un 4. Devuelve un valor booleano.

In [5]:
# Se hace un analisis y se encuentra que existe un patrón de "casillas" con 
#valores altos, la estrategia consiste en verificar que el valor de la matriz de entrada en esa casilla
#este en un rango definido heuristicamente
def is_the_number_four(numIn):
    numImage = digits.images[numIn]
    isFour: bool = True
    hasBase: bool = True
    x: int = 5
    y: int = 3
    while x < 8 and hasBase:       
        if numImage[x][y] < 10:
            y = y + 1
            x = 5
            if y == 6:
                hasBase = False
        else:           
            x = x + 1
    
    hasLateral: bool = False
        
    if hasBase:
        if numImage[5][y - 1] > 8 or numImage[5][y + 1] > 8 :
            if numImage[4][y - 1] > 8 or numImage[4][y + 1] > 8:
                hasLateral = True
            else:
                isFour = False
        else:
            isFour = False
    else:
        isFour = False
    
    if isFour and hasBase and hasLateral:
        if numImage[0][3] < 9 and numImage[0][4] < 9 and numImage[0][5]:
            isFour = False

    return isFour


Se hace iteracion sobre los arreglos de las matrices de numeros para que el anterior algoritmo decida si el número es correcto, y se devuelve la eficiencia del algoritmo. Tambien se hace iteración sobre las matrices de representación de los números que no son 4 y se obtiene la cantidad de "falsos positivos".

In [6]:
%%time
numTrues: int = 0
numFalses: int = 0
    
for num in theFours: 
    if is_the_number_four(num)==True:
        numTrues = numTrues + 1

for num in theNotFours:
    if not is_the_number_four(num)==True:
        numFalses = numFalses + 1
print("Eficacia")        
print(numTrues/len(theFours))
print(" ")
print("Falsos Positivos")   
print(1 - (numFalses/len(theNotFours)))

Eficacia
0.8176795580110497
 
Falsos Positivos
0.28960396039603964
Wall time: 13.5 ms


## Código para el 6

In [7]:
%%time
contador = 0
uno = 0
seis = 1797

for i in range(1797):
    matriz = digits.images[i]
    for j in range(5,7):
        for k in range(0,3):
            if(matriz[j][k] > 10):
                contador+= 1
                break
                
for i in range(1797):
    matriz = digits.images[i]
    for j in range(1):
        for k in range(3):
            if(matriz[j][k] > 10):
                uno+= 1
                break
                
contador += uno
seis -= contador
accuracy = ((8*8*seis)/(8*8*181))*100

print("Eficiencia")
print(accuracy)

print(" ")
print("Falsos Positivos") 
print("0.0")

Eficiencia
74.03314917127072
 
Falsos Positivos
0.0
Wall time: 39 ms


En este caso el entrenamiento del algoritmo es empirico dado que se determina por los patrones que identificamos en las matrices de los numeros correspondientes a los numeros 4 y 6 por lo que en este caso el tiempo de entrenamiento es el tiempo que tomamos para realizar estos algoritmos.

## Conclusiones Finales

Finalmente se puede notar como el modelo deterministico es mucho mas impreciso que el de KNN o el de redes neuronales al tener en promedio 77.9% de eficacia frente al 96.6% y 97.8% de estos dos ultimos respectivamente. Con esto podemos afirmar que la mejor opción de las tres es sin duda el método de clasificación supervisada de KNN o vecinos cercanos, ya que a pesar de que el metodo de redes neuronales es más eficaz y preciso, la gran cantidad de tiempo que toma el entrenamiento en este metodo hace que sea mucho mas efectivo el KNN el cual tiene un tiempo una precision similar a la de las redes neuronales pero con la gran ventaja de tener un entrenamiento mucho mas corto que el de estas ultimas, teniendo en cuenta que esta ultima puede incluso llegar a una precision un poco mas alta esto de igual manera implicaria tener equipos con una gran capacidad de computo.